Testing notebook for different modules of the codebase
#create config object
#import classes/methods to run
#visualise output

In [10]:
#set imports for the package structure
import sys
import os 

# add src code dir to path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

# import my modules
try:
    from configs.config1 import Config
except ImportError as e:
    print(f"Error importing Config: {e}")
